## 목차
### 1. Library Import
### 2. 학습데이터 생성
### 3. Light-gbm 모델 훈련
### 4. 교차 검증 점수 확인
### 5. 제출 파일 생성

# 1. Library Import

In [2]:
!pip install --upgrade pip

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\sue_b\\AppData\\Local\\Temp\\pip-uninstall-vnwphe1k\\pip.exe'
Consider using the `--user` option or check the permissions.




  Using cached pip-21.0-py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.3.3
    Uninstalling pip-20.3.3:
      Successfully uninstalled pip-20.3.3


In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc #garbage collection 감시
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
import seaborn as sns
from functools import partial
from bayes_opt import BayesianOptimization
from lightgbm import LGBMClassifier

# 3. Light-gbm 모델 훈련

In [4]:
train = pd.read_csv("Jupyter Temp/lg 공모전/0126 data/train.csv")
test = pd.read_csv("Jupyter Temp/lg 공모전/0126 data/test.csv")

In [5]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x =train.drop(columns=["label","user_id"])
train_y = train["label"]

test = test.drop(columns="user_id")

print(train_x.shape)
print(train_y.shape)

(15000, 69)
(15000,)


In [19]:
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True

In [22]:
def lgbm_cv(colsample_bytree, 
            learning_rate,
            max_depth,
            n_estimators,
            num_leaves,
          x_data=None, y_data=None, n_splits=5, output='score'):
    
    score = 0
    kf = KFold(n_splits=n_splits, random_state=1029, shuffle=True)
    models = []
    for train_index, valid_index in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_index], y_data.iloc[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data.iloc[valid_index]

        model = LGBMClassifier(
            colsample_bytree = colsample_bytree,
            learning_rate = learning_rate,
            max_depth = int(max_depth),
            n_estimators = int(n_estimators),
            num_leaves = int(num_leaves),
            application = "binary"
            
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        valid_prob = model.predict(x_valid)
        valid_pred = np.where(valid_prob > 0.5, 1, 0)
        
        recall    = recall_score(    y_valid, valid_pred)
        precision = precision_score( y_valid, valid_pred)
        auc_score = roc_auc_score(   y_valid, valid_prob)
    
    if output == 'score':
        return auc_score
    if output == 'model':
        return models

In [ ]:
func_fixed = partial(lgbm_cv, x_data=train_x, y_data=train_y, n_splits=5, output="score")
regBO = BayesianOptimization(
    func_fixed, 
    {
         'colsample_bytree' : (0.3,1),
         'learning_rate':(0.01, 0.1),
         "max_depth" : (200,1000),
         "n_estimators" : (30,50),
         "num_leaves" : (300,1000),
    }, random_state=3424)

regBO.maximize(init_points=1000, n_iter=0)

|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | num_le... |
-------------------------------------------------------------------------------------
|  1        |  0.7215   |  0.8197   |  0.09961  |  456.1    |  48.9     |  869.0    |
|  2        |  0.7245   |  0.8322   |  0.09554  |  925.0    |  37.94    |  305.1    |
|  3        |  0.6605   |  0.7473   |  0.01124  |  234.5    |  41.25    |  313.3    |
|  4        |  0.7191   |  0.6984   |  0.04964  |  438.7    |  31.15    |  964.2    |
|  5        |  0.678    |  0.7033   |  0.01709  |  789.3    |  33.52    |  800.9    |
|  6        |  0.7173   |  0.3512   |  0.08345  |  700.9    |  42.52    |  861.6    |
|  7        |  0.6991   |  0.853    |  0.02462  |  697.0    |  30.63    |  602.1    |
|  8        |  0.7202   |  0.6542   |  0.04384  |  891.2    |  42.65    |  805.5    |
|  9        |  0.7276   |  0.9241   |  0.0723   |  390.3    |  45.17    |  342.3    |
|  10       |  0.6924   |  0.3002   |  0.03158  |  433

In [ ]:
def catboost_eval(bagging_temperature ,
                  depth , 
                  learning_rate ,
                  min_data_in_leaf , 
                  max_leaves , 
                  l2_leaf_reg , 
                  border_count):
    n_splits=5
    kf = KFold(n_splits=n_splits, random_state=1029, shuffle=True)
    score = []
    predict = None
    params = {}
    params['iterations'] = 1000
    params['custom_loss'] = 'TotalF1'
    params['eval_metric'] = 'TotalF1'
    params['random_seed'] = 1234
    params['learning_rate'] = learning_rate
    params['min_data_in_leaf'] = int(round(min_data_in_leaf))
    params['depth'] = int(round(depth))
    params['max_leaves'] = int(round(max_leaves))
    params['l2_leaf_reg'] = int(round(l2_leaf_reg))
    params['border_count'] = int(round(border_count))
    params['bagging_temperature'] = int(round(bagging_temperature))
    
    X , y = catX_train.values , caty_train
    for train_index, valid_index in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_index], y_data.iloc[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data.iloc[valid_index]
        
        clf = CatBoostClassifier(**params , 
                         leaf_estimation_iterations = 10,
                         use_best_model=True,
                         od_type="Iter",
                         logging_level='Silent',
                        )
        clf.fit(X_train, 
                y_train,
                cat_features=cat_features,
                eval_set=(X_valid, y_valid),
                verbose = False ,
        )
        
        valid_prob = model.predict(x_valid)
        valid_pred = np.where(valid_prob > 0.5, 1, 0)
        
        recall    = recall_score(    y_valid, valid_pred)
        precision = precision_score( y_valid, valid_pred)
        auc_score = roc_auc_score(   y_valid, valid_prob)
        
        score.append(auc_score)
        
    return sum(score)/n_splits

## min_data_in_leaf , max_leaves , 추가 하니 먼가 잘 안됨.
catBO = BayesianOptimization(catboost_eval,
                             {'bagging_temperature': (0, 1000),
                              'depth': (5, 8.99) ,
                              "learning_rate" : (0.001,0.1) , 
                              "min_data_in_leaf" : (1,6) , 
                              'max_leaves' : (200,200)  ,
                              'l2_leaf_reg': (100, 100)  ,
                              'border_count': (5, 255) ,
                             },
                             random_state=0)
init_round=5
opt_round = 10
catBO.maximize(init_points=init_round, n_iter=opt_round)

# 4. 교차검증 점수 확인

In [26]:
params = regBO.max['params']

0.8029743085655111


In [ ]:
models = lgbm_cv(
    params['colsample_bytree'], 
    params['learning_rate'],
    params["max_depth"],
    params["n_estimators"],
    params["num_leaves"],
    x_data=train_x, y_data=train_y, n_splits=5, output='model')

# 5. 제출 파일 생성

In [27]:
# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict(test)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [28]:
pred_ensemble

array([[0.6505471 ],
       [0.27931295],
       [0.31776057],
       ...,
       [0.38540448],
       [0.65914218],
       [0.30982861]])

In [29]:
sample_submssion = pd.read_csv("Jupyter Temp/lg 공모전/data/sample_submission.csv")

In [30]:
sample_submssion['problem'] = pred_ensemble.reshape(-1)

In [31]:
sample_submssion.to_csv("Jupyter Temp/lg 공모전/submissions/0127(1).csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.650547
1,30001,0.279313
2,30002,0.317761
3,30003,0.596035
4,30004,0.676201
...,...,...
14994,44994,0.307844
14995,44995,0.342251
14996,44996,0.385404
14997,44997,0.659142


In [32]:
len(sample_submssion)

14999

In [16]:
!pip install pycaret

You should consider upgrading via the 'c:\users\sue_b\anaconda3\python.exe -m pip install --upgrade pip' command.


In [17]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [33]:
col = list(train_x.columns)

In [35]:
auto_train = train.drop(columns="user_id")

In [36]:
clf = setup(data = auto_train, target = 'label', numeric_features=col)

,Description,Value
0,session_id,1112
1,Target,label
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 70)"
5,Missing Values,False
6,Numeric Features,69
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [37]:
best_5 = compare_models(sort = 'Accuracy', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.7935,0.8222,0.5201,0.7955,0.6285,0.4942,0.5159,22.4020
gbc,Gradient Boosting Classifier,0.7917,0.8177,0.4932,0.8148,0.6139,0.4831,0.5118,3.1490
lightgbm,Light Gradient Boosting Machine,0.7914,0.8141,0.5280,0.7818,0.6297,0.4922,0.5108,1.1100
rf,Random Forest Classifier,0.7886,0.8169,0.5039,0.7922,0.6155,0.4798,0.5033,2.0980
et,Extra Trees Classifier,0.7886,0.8093,0.5133,0.7842,0.6200,0.4825,0.5035,2.8330
ada,Ada Boost Classifier,0.7836,0.8035,0.4994,0.7789,0.6080,0.4684,0.4907,0.8020
xgboost,Extreme Gradient Boosting,0.7803,0.8020,0.5402,0.7365,0.6230,0.4732,0.4846,3.9620
lda,Linear Discriminant Analysis,0.7678,0.7646,0.3862,0.8349,0.5276,0.4004,0.4528,0.1570
ridge,Ridge Classifier,0.7665,0.0000,0.3794,0.8381,0.5218,0.3955,0.4499,0.0710
qda,Quadratic Discriminant Analysis,0.7514,0.7556,0.3690,0.7745,0.4993,0.3608,0.4048,0.1040


In [38]:
blended = blend_models(estimator_list = best_5, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8024,0.8233,0.5431,0.8067,0.6492,0.5188,0.5385
1,0.7943,0.8288,0.5219,0.7970,0.6308,0.4966,0.5180
2,0.7838,0.8149,0.4837,0.7935,0.6011,0.4645,0.4913
3,0.8014,0.8291,0.5120,0.8341,0.6345,0.5087,0.5373
4,0.7794,0.8144,0.4717,0.7872,0.5899,0.4517,0.4794
Mean,0.7923,0.8221,0.5065,0.8037,0.6211,0.4881,0.5129
SD,0.0092,0.0064,0.0259,0.0165,0.0221,0.0258,0.0239


In [39]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7969,0.8118,0.4939,0.8080,0.6130,0.4860,0.5131


In [40]:
final_model = finalize_model(blended)


In [41]:
predictions = predict_model(final_model, data = test)

In [42]:
x = []
for i in range(len(predictions['Score'])):
    if predictions['Label'][i] =='1.0':
        x.append(predictions['Score'][i])
    else:
        x.append(1-predictions['Score'][i])

In [43]:
sample_submssion = pd.read_csv("Jupyter Temp/lg 공모전/data/sample_submission.csv")
sample_submssion['problem'] = x
sample_submssion.to_csv("Jupyter Temp/lg 공모전/submissions/0127(2).csv", index = False)

In [44]:
sample_submssion

,user_id,problem
0,30000,0.9429
1,30001,0.2141
2,30002,0.3186
3,30003,0.7615
4,30004,0.8329
...,...,...
14994,44994,0.4111
14995,44995,0.3111
14996,44996,0.5184
14997,44997,0.8479
